In [1]:
! pip install -q --upgrade gradio langchain langchain_community openai huggingface_hub datasets langchain_huggingface langchain_openai chromadb faiss-cpu
! pip install -q --upgrade datasets fsspec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.5/725.5 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 22.2 MB/s e

In [ ]:
import os
import openai
import gradio as gr

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

from datasets import load_dataset
from huggingface_hub import login
from google.colab import userdata

In [3]:
# environment setup
openai_api_key = userdata.get('OPENAI_API_KEY')
login(token=userdata.get('HF_TOKEN'))

In [4]:
# CONSTANTS
DB_NAME = 'vector_db'
MODEL = "gpt-3.5-turbo"

In [5]:
ds = load_dataset("enelpol/rag-mini-bioasq", "text-corpus")

README.md:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/35.3M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/40181 [00:00<?, ? examples/s]

In [6]:
documents = [
    Document(
        page_content=entry['passage'],
        metadata={"id": entry.get("id", "N/A")}
        )
    for entry in ds['test']
]

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunks = text_splitter.split_documents(documents)

In [8]:
len(chunks)

79850

In [9]:
embed_model = HuggingFaceEmbeddings(
    model_name="intfloat/e5-base",
    model_kwargs={'device': 'cuda'},
    encode_kwargs={
        'batch_size': 256,
    }
)

<ipython-input-9-15dec62560f4>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/356 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [10]:
db = FAISS.from_documents(chunks, embedding=embed_model)
db.save_local(DB_NAME)

In [11]:
llm = ChatOpenAI(model_name=MODEL, temperature=0,api_key=openai_api_key)

memory = ConversationBufferMemory(memory_key='chat_history',return_message=True)

retriever = db.as_retriever()

chain = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

<ipython-input-11-c4e945180eb6>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history',return_message=True)


In [12]:
query = "What is the difference between Eucaryotic and Procaryotic organisms"
response = chain({"question": query})
print(response)

<ipython-input-12-219cc2aea61f>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain({"question": query})


{'question': 'What is the difference between Eucaryotic and Procaryotic organisms', 'chat_history': '', 'answer': 'Eucaryotic organisms, also known as eukaryotes, have cells that contain a nucleus and other membrane-bound organelles. They are typically more complex and larger than procaryotic organisms. On the other hand, procaryotic organisms, also known as prokaryotes, lack a nucleus and membrane-bound organelles. They are usually smaller and simpler in structure compared to eucaryotic organisms.'}


In [13]:
def chat(history, query):
    response = chain({"question": query})
    return response['answer']

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://edc16111755a217abb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
